In [111]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import time 
import xl2dict
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件


# 找到excel資料存放的位置，將所有檔案的路徑變成list

In [113]:
myxlobject= xl2dict.XlToDict()
dataDict=myxlobject.convert_sheet_to_dict(file_path=pathList[0], sheet="First Sheet")
dataDict[0]

{'個人自願提繳率(%)': '',
 '勞基法特殊身分別': '',
 '勞工保險證號': '05952202',
 '勞工保險證號檢查碼': 'F',
 '勞退提繳日期(與勞保加保日期不同時才需輸入)': '',
 '已領取社會保險給付種類': '',
 '提繳身分別': '1',
 '月實際工資': '11100',
 '格式別': '1',
 '特殊身分別': '1',
 '異動別': '4',
 '被保險人出生日期': '0880224',
 '被保險人外籍': '',
 '被保險人姓名': '吳名豪',
 '被保險人性別': '',
 '被保險人身份證號': 'A130528512',
 '雇主提繳率(%)': '6'}

# 將每個excel轉成dict，並且判斷裡面內容:
### 　　單位-保險證號
### 　　加保、退保、調保
### 　　三合一、二合一

# 讓使用者輸入:
### 　　pin碼
### 　　檔案資料夾位置

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage



try:

    pinNum="780311" #需改成人工輸入

    chrome_options = Options()
    #crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
    chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
    #webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
    #套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
    driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
    driver.get("https://edesk.bli.gov.tw/cpa/")
    
    #找到excel資料存放的位置，將所有檔案的路徑變成list
    dataDir=r"D:\保險申報檔案" #需改成人工輸入
    #遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
    # for i in os.walk(dataDir):
    #     for j in i[2]:
    #         print(i[0]+j)
    #可縮寫成這樣     
    pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2]]
    myxlobject= xl2dict.XlToDict()
    for excelFilePath in pathList:
        #得到excel內容成dict
        dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
        #進入勞保局
        WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
    #     alert=driver.switch_to_alert()
    #     alert.dismiss()
        #關閉阻擋頁
        driver.find_element_by_class_name("b-close").click()
        #輸入pin碼(自然人憑證)
        pinInput=driver.find_element_by_name("pin")
        pinInput.clear()
        pinInput.send_keys(pinNum)#憑證pin碼
        #輸入保險證號
        unoInput=driver.find_element_by_name("uno")
        unoInput.clear()
        unoInput.send_keys(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])#"單位保險證號"
        time.sleep(0.5)
        #確認進入
        driver.find_element_by_name('Submit').click()
        WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
        passOrNot=True
        driver.find_element_by_class_name("b-close").click()
        time.sleep(0.4)
        driver.find_element_by_id("mi0,0").click()
        time.sleep(0.1)
        driver.find_element_by_id("mi0,5").click()
        time.sleep(0.2)

        if dataDict['異動別']=="4":
            driver.find_element_by_id("mi0,6").click()#加保
        elif dataDict['異動別']=="2":
            driver.find_element_by_id("mi0,7").click()#退保
        elif dataDict['異動別']=="3":
            driver.find_element_by_id("mi0,8").click()#調保
        time.sleep(0.8)

        if "健保投保單位代碼" in dataDict:
            driver.find_elements_by_xpath('//*[@id="type"]')[0].click()#三合一
        else:
            driver.find_elements_by_xpath('//*[@id="type"]')[1].click()#二合一

        driver.find_element_by_id("uploadFile").send_keys(excelFilePath)
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="pin"]').send_keys(pinNum)
    #     time.sleep(3)
        # driver.find_element_by_id("btnSaveB").click()
        driver.find_element_by_id("mi0,53").click()
        time.sleep(1)

    driver.close()
except Exception as e:
    msg = EmailMessage()
    msg.set_content("error: "+str(e))
    msg['Subject'] = 'surcure'
    msg['From'] = "lisa"
    msg['To'] = 'andy.yuan@wowprime.com'
    with smtplib.SMTP('192.168.2.1',25) as s:
        s.send_message(msg)

KeyboardInterrupt: 

In [1]:

#  {'個人自願提繳率(%)': '',
#  '勞基法特殊身分別': '',
#  '勞工保險證號': '05952202',
#  '勞工保險證號檢查碼': 'F',
#  '勞退提繳日期(與勞保加保日期不同時才需輸入)': '',
#  '已領取社會保險給付種類': '',
#  '提繳身分別': '1',
#  '月實際工資': '11100',
#  '格式別': '1',
#  '特殊身分別': '1',
#  '異動別': '4',
#  '被保險人出生日期': '0880224',
#  '被保險人外籍': '',
#  '被保險人姓名': '吳名豪',
#  '被保險人性別': '',
#  '被保險人身份證號': 'A130528512',
#  '雇主提繳率(%)': '6'}

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ANDY~1.YUA\AppData\Local\Temp\jieba.cache
Loading model cost 1.040 seconds.
Prefix dict has been built succesfully.


['你', '是', '天才']

# 最新進度

In [176]:
def selectPath():
    path_ = askdirectory()
    path.set(path_)
def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)
    
    
with open(r'D:\保險申報檔案\試打包\defaultIO.txt') as f:
    IO=f.read()
defaultLoadDir=IO.split("\n")[0].split("=")[1]
defaultSaveDir=IO.split("\n")[1].split("=")[1]
root = Tk()
path = StringVar()
Label(root,text = "讀取資料夾:").grid(row = 0, column = 0)
e1=Entry(root, textvariable = path)
e1.insert(0, defaultLoadDir)
e1.grid(row = 0, column = 1)
Button(root, text = "讀取資料夾", command = selectPath).grid(row = 0, column = 2)

path2 = StringVar()
Label(root,text = "存放資料夾:").grid(row = 1, column = 0)
e3=Entry(root, textvariable = path2)

e3.insert(0, defaultSaveDir)
e3.grid(row = 1, column = 1)

Button(root, text = "存放資料夾", command = selectPath2).grid(row = 1, column = 2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column = 0)
e2 = Entry(root)
e2.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='加退調保開始').grid(row=3, column=1, pady=4)
root.mainloop()

In [159]:
pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2]]
print([i.split("\\")[-2] for i in pathList])
for i in pathList:
    print(i)



['test', 'test', 'test', 'test', '0211', '0211', '1070210', '1070210']
D:\保險申報檔案\test\11801_調1二合一.xls
D:\保險申報檔案\test\11802_調2.xls
D:\保險申報檔案\test\投保號碼改掉sample.xls
D:\保險申報檔案\test\改投保.xls
D:\保險申報檔案\test\0211\11801_加2二合一.xls
D:\保險申報檔案\test\0211\11801_退1.xls
D:\保險申報檔案\test\1070210\11802_加2二合一.xls
D:\保險申報檔案\test\1070210\11802_退2二合一.xls


In [163]:
pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2]]
for excelFilePath in pathList:
    print(excelFilePath)
    dirName=excelFilePath.split("\\")[-2]
    print(dirName)

D:\保險申報檔案\test\11801_調1二合一.xls
test
D:\保險申報檔案\test\11802_調2.xls
test
D:\保險申報檔案\test\投保號碼改掉sample.xls
test
D:\保險申報檔案\test\改投保.xls
test
D:\保險申報檔案\test\0211\11801_加2二合一.xls
0211
D:\保險申報檔案\test\0211\11801_退1.xls
0211
D:\保險申報檔案\test\1070210\11802_加2二合一.xls
1070210
D:\保險申報檔案\test\1070210\11802_退2二合一.xls
1070210


# 一步一步測試區

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage
import xlrd

pinNum="780311" #需改成人工輸入

chrome_options = Options()
#crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
#webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
#套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
driver.get("https://edesk.bli.gov.tw/cpa/")

#找到excel資料存放的位置，將所有檔案的路徑變成list
dataDir=r"D:\保險申報檔案\保險申報檔案\11802" #需改成人工輸入
#遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
# for i in os.walk(dataDir):
#     for j in i[2]:
#         print(i[0]+j)
#可縮寫成這樣     
pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2]]
myxlobject= xl2dict.XlToDict()
excelFilePath=pathList[0]
#得到excel內容成dict
dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
#進入勞保局
WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
#     alert=driver.switch_to_alert()
#     alert.dismiss()
#關閉阻擋頁
driver.find_element_by_class_name("b-close").click()
#輸入pin碼(自然人憑證)
pinInput=driver.find_element_by_name("pin")
pinInput.clear()
pinInput.send_keys(pinNum)#憑證pin碼
#輸入保險證號
unoInput=driver.find_element_by_name("uno")
unoInput.clear()

print(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼']+dataDict['異動別'])
unoInput.send_keys(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])#"單位保險證號"
uNoList=set()
uNoList.add(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
time.sleep(0.5)
#確認進入
driver.find_element_by_name('Submit').click()
WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
passOrNot=True
driver.find_element_by_class_name("b-close").click()
time.sleep(0.4)
driver.find_element_by_id("mi0,0").click()
time.sleep(0.1)
driver.find_element_by_id("mi0,5").click()
time.sleep(0.2)

print(dataDict['異動別'])
if dataDict['異動別']=="4":
    driver.find_element_by_id("mi0,6").click()#加保
elif dataDict['異動別']=="2":
    driver.find_element_by_id("mi0,7").click()#退保
elif dataDict['異動別']=="3":
    driver.find_element_by_id("mi0,8").click()#調保
time.sleep(0.8)

if "健保投保單位代碼" in dataDict:
    driver.find_elements_by_xpath('//*[@id="type"]')[0].click()#三合一
else:
    driver.find_elements_by_xpath('//*[@id="type"]')[1].click()#二合一

driver.find_element_by_id("uploadFile").send_keys(excelFilePath)
time.sleep(0.5)
driver.find_element_by_xpath('//*[@id="pin"]').send_keys(pinNum)
#     time.sleep(3)
driver.find_element_by_id("btnSaveB").click()

def success():
    try:
        return '資料新增成功' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text
    except:
        return False
while not success():
    time.sleep(0.5)


05952199X4


UnexpectedAlertPresentException: Alert Text: None
Message: unexpected alert open: {Alert text : 讀卡機沒插卡}
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 6.1.7601 SP1 x86_64)


In [78]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage
import xlrd

pinNum="780311" #需改成人工輸入

chrome_options = Options()
#crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
#webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
#套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
driver.get("https://edesk.bli.gov.tw/cpa/")

#找到excel資料存放的位置，將所有檔案的路徑變成list
dataDir=r"D:\保險申報檔案\test" #需改成人工輸入
#遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
# for i in os.walk(dataDir):
#     for j in i[2]:
#         print(i[0]+j)
#可縮寫成這樣     
pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2]]
myxlobject= xl2dict.XlToDict()
excelFilePath=pathList[0]
#得到excel內容成dict
dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
#進入勞保局
WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
#     alert=driver.switch_to_alert()
#     alert.dismiss()
#關閉阻擋頁
time.sleep(3)
driver.find_element_by_class_name("b-close").click()
#輸入pin碼(自然人憑證)
pinInput=driver.find_element_by_name("pin")
pinInput.clear()
pinInput.send_keys(pinNum)#憑證pin碼
#輸入保險證號
unoInput=driver.find_element_by_name("uno")
unoInput.clear()

print(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼']+dataDict['異動別'])
unoInput.send_keys(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])#"單位保險證號"
uNoList=set()
uNoList.add(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
time.sleep(0.5)
#確認進入
driver.find_element_by_name('Submit').click()
time.sleep(0.5)


05952202F3


In [74]:
pathList=[i[0]+"/"+j for i in os.walk(dataDir) for j in i[2]]

for path in pathList:
    dirName=path.split("\\")[-1].split("/")[0]
#     print(dirName)
    if len(dirName)==7:#民國年
        choice=dirName[:3]+"/"+dirName[3:5]+'/'+dirName[5:] 
    elif len(dirName)==8:#西元年
        try:
            choice=str(int(dirName[:4])-1911)+"/"+dirName[4:6]+'/'+dirName[6:]
        except:
            pass
    elif len(dirName)==4:#只有月日
        try:
            choice=str(int(time.strftime("%Y"))-1911) +"/"+dirName[0:2]+'/'+dirName[2:]
        except:
            pass
#     print(choice)
    try:
        effDateOption=driver.find_element_by_name("laefDte")
        effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
        for date in effDate:
            if choice in date:
                driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                time.sleep(0.2)
#                 print(date+"\n")
    except:
        pass
    

107/02/11(日)

107/02/11(日)

107/02/10(六)

107/02/10(六)

107/02/10(六)

107/02/10(六)

107/02/11(日)

107/02/11(日)

107/02/11(日)

107/02/11(日)



In [32]:
driver.find_element_by_name("laefDte").click()

In [43]:
effDateOption=driver.find_element_by_name("laefDte")
effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
for date in effDate:
    driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()

In [136]:
#########################################################被保名冊下載
time.sleep(0.3)
driver.find_element_by_id("mi0,24").click()
time.sleep(0.2)
driver.find_element_by_id("mi0,26").click()
time.sleep(0.4)

try:
    target =driver.find_element_by_xpath("//input[@value='查  詢']").click()
    # driver.execute_script("arguments[0].scrollIntoView();", target)
    # target.click()
    time.sleep(0.5)
    effDateOption=driver.find_element_by_id("format")
    effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
    for date in effDate:
        if "含保費" in date:
            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
            time.sleep(0.2)
    time.sleep(0.8)
    driver.find_element_by_name("Submit722").click()
except:
    pass
#勞退下載
try:
    time.sleep(0.3)
    driver.find_element_by_id("mi0,24").click()
    time.sleep(0.2)
    driver.find_element_by_id("mi0,34").click()
    time.sleep(0.4)
    driver.find_element_by_name("Submit").click()
    time.sleep(0.4)
    effDateOption=driver.find_element_by_id("hiddenidn")
    effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
    for date in effDate:
        if "全部顯示" in date:
            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
            time.sleep(0.2)
    driver.find_element_by_xpath("//input[@value='TXT下載']").click()
    time.sleep(0.2)
    driver.back()
except:
    pass

try:
    effDateOption=driver.find_element_by_id("accountType")
    effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
    for date in effDate:
        if "個人" in date:
            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
            time.sleep(0.2)
    driver.find_element_by_name("Submit").click()
    time.sleep(0.4)
    effDateOption=driver.find_element_by_id("hiddenidn")
    effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
    for date in effDate:
        if "全部顯示" in date:
            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
            time.sleep(0.2)
    driver.find_element_by_xpath("//input[@value='TXT下載']").click()
    time.sleep(0.2)
except:
    pass
#############################################################

In [9]:
import sys

errorMsg=""
report=[]
try:
    try:
        WebDriverWait(driver,3).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
        time.sleep(0.2)
        driver.find_element_by_class_name("b-close").click()
    except:
        report.append("登入失敗-異動別:{}\t證號:{}\t位置{}")
        time.sleep(0.3)
        alert=driver.switch_to_alert()
        ###############################################需增加判斷密碼錯誤、權限錯誤問題
        if '讀卡機' in alert.text:
            errorMsg="讀卡機沒插卡"
            driver.quit()
            sys.exit("RRR")
        elif '權限' in alert.text:
            errorMsg="你沒權限跑{}".format(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
            driver.quit()
            sys.exit("RRR")
        elif '密碼' in alert.text:
            errorMsg="你密碼錯誤"
            driver.quit()
            sys.exit("RRR")

        print('safsfadfas')

        ###############################################
    print(errorMsg+"aaa")
except:
    print(errorMsg)
    
# try:
#     alert=driver.switch_to_alert()
    
# except:
#     print('xxxxx')

讀卡機沒插卡


In [8]:
import xlrd
path=r"D:\保險申報檔案\存放區\sample"
for file_ in os.listdir(path):
    name=[]
    book = xlrd.open_workbook(os.path.join(path, file_))
    sheet=book.sheet_by_index(0)
    
    #日期
    dateCell=sheet.cell_value(rowx=3,colx=8) or sheet.cell_value(rowx=3,colx=4)
    if len(dateCell.split("／"))<3:
        name.append(dateCell.split("：")[1].split(" ")[0])
    else:
        name.append(dateCell.split("／")[2] or dateCell.split("／")[1].split("：")[1].split(" ")[0])

    ##事業部
    if "公司" in sheet.cell_value(rowx=4,colx=0):
        name.append(sheet.cell_value(rowx=4,colx=0).split("公司")[1] or sheet.cell_value(rowx=3,colx=0).split("：")[1])
    else:
        name.append(sheet.cell_value(rowx=3,colx=0).split("公司")[1] or sheet.cell_value(rowx=2,colx=0).split("：")[1])
    
    
    ##加退調保
    if '加' in sheet.cell_value(rowx=0,colx=0):
        name.append("加")
    elif '退' in sheet.cell_value(rowx=0,colx=0):
        name.append("退")
    elif '調' in sheet.cell_value(rowx=0,colx=0):
        name.append("調")
        
    ##二、三合一
    if '健' in sheet.cell_value(rowx=1,colx=0):
        name.append("3")
    else:
        name.append("2")
    
    try:
        os.rename(os.path.join(path,file_),os.path.join(path,"_".join(name)+".xls"))
    except:
        print(os.path.join(path, file_)+"\t已有變更檔名的檔案")

D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208 (3).xls	已有變更檔名的檔案
D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208 (6).xls	已有變更檔名的檔案
D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208 (7).xls	已有變更檔名的檔案
D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208 (8).xls	已有變更檔名的檔案
D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208 (9).xls	已有變更檔名的檔案
D:\保險申報檔案\存放區\sample\CaApplyBatchData_1070208.xls	已有變更檔名的檔案


In [49]:
sheetttt=book.sheet_by_name("本人")

In [77]:
x=["",'aaa']
for i in x:
    print(i or "xx")

xx
aaa


In [ ]:
os.rename(os.path.join(path,file_),os.path.join(path,"_".join(name)+".xls"))

# 自動加退調保

In [ ]:
from tkinter import Tk,StringVar,Label,Entry,Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait#直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import sys
import time 
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage



uNoList=set()
report=[]
changDict={"4":"加保","2":"退保","3":"調保"}
web='https://edesk.bli.gov.tw/cpa/'
    
with open('Email.txt') as f:
    mails=f.read().split('\n')
while "" in mails:
    mails.remove("")
mails=set(mails)

def mailTo(title,mailAdds,message,whoSend='AutoAddInsurance'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()        
    
    


def doAddPopChangeAssure():
    path2=e3.get()
    unoNumber=0
    excelFilePath=""
    def renamefile(path):
#         pathList2=[i[0]+"/"+j for i in os.walk(path) for j in i[2]]
        for pathTuple in os.walk(path):
            for file_ in pathTuple[2]:
                try:
                    if 'xls' in file_ and 'crdownload' not in file_:
                        name=[]
                        book = xlrd.open_workbook(os.path.join(pathTuple[0], file_))
                        sheet=book.sheet_by_index(0)

                        #日期
                        dateCell=sheet.cell_value(rowx=3,colx=8) or sheet.cell_value(rowx=3,colx=4)
                        if len(dateCell.split("／"))<3:
                            name.append(dateCell.split("：")[1].split(" ")[0])
                        else:
                            name.append(dateCell.split("／")[2] or dateCell.split("／")[1].split("：")[1].split(" ")[0])

                        ##事業部
                        if "公司" in sheet.cell_value(rowx=4,colx=0):
                            name.append(sheet.cell_value(rowx=4,colx=0).split("公司")[1] or sheet.cell_value(rowx=3,colx=0).split("：")[1])
                        else:
                            name.append(sheet.cell_value(rowx=3,colx=0).split("公司")[1] or sheet.cell_value(rowx=2,colx=0).split("：")[1])


                        ##加退調保
                        if '加' in sheet.cell_value(rowx=0,colx=0):
                            name.append("加")
                        elif '退' in sheet.cell_value(rowx=0,colx=0):
                            name.append("退")
                        elif '調' in sheet.cell_value(rowx=0,colx=0):
                            name.append("調")

                        ##二、三合一
                        if '健' in sheet.cell_value(rowx=1,colx=0):
                            name.append("3")
                        else:
                            name.append("2")

                        try:
                            os.rename(os.path.join(pathTuple[0],file_),os.path.join(pathTuple[0],"_".join(name)+".xls"))
                        except:
    #                         print(os.path.join(path, file_)+"\t已有變更檔名的檔案")
                            os.remove(os.path.join(pathTuple[0],file_))
                except:
                    report.append("更名失敗>>{}".format(os.path.join(pathTuple[0],file_)))


    try:
        uNoList=set()
        report=[]
        b=time.time()
        errorMsg=""
        pinNum=e2.get() #pin號碼
        chrome_options = Options()
        #crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
        chrome_options.add_extension('extension_0_0_1_3.crx')
        
        chrome_options.add_experimental_option("prefs", {
          "download.default_directory": e3.get(),
          "download.prompt_for_download": False,
          "download.directory_upgrade": True,
          "safebrowsing.enabled": True
        })
        
        #webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
        #套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
        driver =webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
        driver.get(web)
        
        dataDir=e1.get() #需改成人工輸入
        #遍歷資料夾內所有檔案  os.walk(路徑位置) 會產生該位置所有的 (路徑,資料夾名稱,檔案名稱)   (包含資料夾內的資料夾內的資料夾....的檔案)
        # for i in os.walk(dataDir):
        #     for j in i[2]:
        #         print(i[0]+j)
        #可縮寫成這樣     
        pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2] if 'xls' in j]
        myxlobject= xl2dict.XlToDict()
        fileCount=0
        for excelFilePath in pathList:
            try:
                #得到excel內容成dict
                dataDict=myxlobject.convert_sheet_to_dict(file_path=excelFilePath, sheet="First Sheet")[0]
                #進入勞保局
                WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
                time.sleep(0.4)
            #     alert=driver.switch_to_alert()
            #     alert.dismiss()
                #關閉阻擋頁
                successful = 0
                sumN = 0
                while successful == 0:
                    try:
                        sumN += 1
                        if sumN > 20:
                            break
                        driver.find_element_by_class_name("b-close").click()
                        successful = 1
                    except:
                        time.sleep(0.3)
                time.sleep(0.2)


                #輸入pin碼(自然人憑證)
                pinInput=driver.find_element_by_name("pin")
                pinInput.clear()
                pinInput.send_keys(pinNum)#憑證pin碼
                #輸入保險證號
                unoInput=driver.find_element_by_name("uno")
                unoInput.clear()
                unoNumber=dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼']#"單位保險證號"
                unoInput.send_keys(unoNumber)
                uNoList.add(unoNumber)
                WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.NAME,'Submit')))
                time.sleep(0.5)
                #確認進入
                successful=0
                sumN=0
                while successful==0:
                    try:
                        sumN+=1
                        if sumN>100:
                            break
                        driver.find_element_by_name('Submit').click()
                        successful = 1
                    except:
                        time.sleep(0.3)
                time.sleep(2)
                try:
                    WebDriverWait(driver,8).until(EC.element_to_be_clickable((By.CLASS_NAME,'b-close')))
                    time.sleep(0.4)
                    # driver.find_element_by_class_name("b-close").click()
                    successful = 0
                    sumN = 0
                    while successful == 0:
                        try:
                            sumN += 1
                            if sumN > 20:
                                break
                            driver.find_element_by_class_name("b-close").click()
                            successful = 1
                        except:
                            time.sleep(0.3)
                    time.sleep(0.4)
                except:
                    report.append("登入失敗-異動別:{}\t證號:{}\t位置{}".
                                      format(changDict[dataDict['異動別']],
                                             dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                             excelFilePath))
                    time.sleep(0.3)
                    alert=driver.switch_to_alert()
                    ###############################################判斷密碼錯誤、權限錯誤問題
                    if '讀卡機' in alert.text:
                        errorMsg="讀卡機沒插卡"
                    elif '權限' in alert.text:
                        errorMsg="你沒權限跑{}".format(dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'])
                    elif '密碼' in alert.text:
                        errorMsg="密碼錯誤>>你所輸入的密碼是:{}".format(pinNum)

                    if errorMsg!="":
                        mailTo('ReportError',mails,errorMsg)
                        driver.quit()
                        root.destroy()
                        sys.exit("因故中斷:"+errorMsg)
                    ###############################################
                    alert.accept()
                    continue

                WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.ID,'mi0,0')))
                time.sleep(0.6)
                successful = 0
                sumN = 0
                while successful == 0:
                    try:
                        sumN += 1
                        if sumN > 100:
                            break
                        driver.find_element_by_id("mi0,0").click()
                        successful = 1

                    except:
                        time.sleep(0.3)
                time.sleep(0.2)
                driver.find_element_by_id("mi0,5").click()
                WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.ID,'mi0,7')))
                time.sleep(0.4)

                if dataDict['異動別']=="4":
                    driver.find_element_by_id("mi0,6").click()#加保
                elif dataDict['異動別']=="2":
                    driver.find_element_by_id("mi0,7").click()#退保
                elif dataDict['異動別']=="3":
                    driver.find_element_by_id("mi0,8").click()#調保
                time.sleep(1)
                #########################################################加入生效日(連假後才需選擇)
                print(excelFilePath)
                dirName=excelFilePath.split("\\")[-2]
                print(dirName)
                if len(dirName)==7:#民國年
                    choice=dirName[:3]+"/"+dirName[3:5]+'/'+dirName[5:]
                elif len(dirName)==8:#西元年
                    try:
                        choice=str(int(dirName[:4])-1911)+"/"+dirName[4:6]+'/'+dirName[6:]
                    except:
                        pass
                elif len(dirName)==4:#只有月日
                    try:
                        choice=str(int(time.strftime("%Y"))-1911) +"/"+dirName[0:2]+'/'+dirName[2:]
                    except:
                        pass
                try:
                    effDateOption=driver.find_element_by_name("laefDte")
                    effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
                    for date in effDate:
                        if choice in date:
                            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            time.sleep(0.2)
                            print(date+"\n"+'*'*50)
                except:
                    pass

                #########################################################
                if "健保投保單位代碼" in dataDict:
                    driver.find_elements_by_xpath('//*[@id="type"]')[0].click()#三合一
                else:
                    driver.find_elements_by_xpath('//*[@id="type"]')[1].click()#二合一

                driver.find_element_by_id("uploadFile").send_keys(excelFilePath)#上傳檔案
                time.sleep(0.5)
                driver.find_element_by_xpath('//*[@id="pin"]').send_keys(pinNum)#輸入pin
                time.sleep(0.2)
                driver.find_element_by_id("btnSaveB").click()
                #睡到成功為止或過了15秒
                def success():
                    try:
                        return '資料新增成功' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text
                    except:
                        return False
                nsec=0
                while (not success()):
                    try:
                        if '失敗' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text:
                            report.append("上傳失敗-異動別:{}\t證號:{}\t位置{}".
                                          format(changDict[dataDict['異動別']],
                                                 dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                                 excelFilePath))
                            try:
                                driver.find_element_by_id("mi0,53").click()
                            except:
                                driver.find_element_by_id("mi0,52").click()
                            continue
                        if '類型錯誤' in driver.find_element_by_xpath('//*[@id="ifon"]/span[2]/font').text:
                            report.append("上傳失敗(超過30筆?)-異動別:{}\t證號:{}\t位置{}".
                                          format(changDict[dataDict['異動別']],
                                                 dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
                                                 excelFilePath))
                            continue
                    except:
                        pass
                    nsec+=0.5
                    if nsec>15:
                        break
                    time.sleep(0.5)
    #                 print(1)
                if success():
                    print("資料上傳成功-異動別:{}\t證號:{}\t位置{}".format(changDict[dataDict['異動別']],dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],excelFilePath))
    #                 report.append("資料上傳成功-異動別:{}\t證號:{}\t位置{}".
    #                                   format(changDict[dataDict['異動別']],
    #                                          dataDict['勞工保險證號']+dataDict['勞工保險證號檢查碼'],
    #                                          excelFilePath))

                time.sleep(0.5)
            except Exception as e:
                report.append("上傳執行失敗-異動別:{}\t證號:{}\t位置{}\n錯誤訊息:{}\t錯誤位置:{}".
                              format(changDict[dataDict['異動別']],
                                     dataDict['勞工保險證號'] + dataDict['勞工保險證號檢查碼'],
                                     excelFilePath,e,sys.exc_info()[2].tb_lineno))
                driver.get(web)
                time.sleep(0.5)
                continue
        #########################################################下載區!!!

            try:
                WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.ID,'mi0,0')))
                time.sleep(0.5)

                successful = 0
                sumN = 0
                while successful == 0:
                    try:
                        sumN += 1
                        if sumN > 100:
                            break
                        driver.find_element_by_id("mi0,0").click()
                        successful = 1
                    except:
                        time.sleep(0.3)
                time.sleep(0.1)
                driver.find_element_by_id("mi0,10").click()
                time.sleep(0.2)
                driver.find_element_by_xpath('//*[@id="main"]/form/fieldset/table/tbody/tr[7]/td[2]/input[2]').click()
                time.sleep(0.2)
                driver.find_element_by_name('btnQuery').click()
                time.sleep(0.5)

                downloadDict={"4":'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchPayList',
                            "2":'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchStopList',
                            "3":'https://edesk.bli.gov.tw/cpa/caApplyDataQueryDispatch.do?method=batchAdjList'}


                driver.get(downloadDict[dataDict['異動別']])
                try:
                    time.sleep(0.2)
                    WebDriverWait(driver,12).until(EC.element_to_be_clickable((By.NAME,'btnBackQueryForm')))
                    time.sleep(0.2)
                    if len(driver.find_elements_by_name('meslamIdOfPrint'))>0:
                        point=driver.find_elements_by_name('meslamIdOfPrint')[0]
                        point.click()
                        fileCount+=1
                        time.sleep(0.2)
                        driver.find_element_by_id('btnPrint2').click()
                    time.sleep(0.6)
                    if fileCount%50==0:
                        renamefile(path2)
                    time.sleep(0.8)
                except Exception as e:
                    time.sleep(0.5)
                    if dataDict['異動別']=='4':
                        doType = "加保"
                    elif dataDict['異動別']=='2':
                        doType = "退保"
                    elif dataDict['異動別']=='1':
                        doType = "調保"

                    report.append("下載失敗-證號:{},種類:{},錯誤訊息:{},錯誤位置:{}".format(dataDict['勞工保險證號'],doType, e,sys.exc_info()[2].tb_lineno))
                time.sleep(0.4)
                # #########################################################被保名冊下載
                # driver.find_element_by_id("mi0,24").click()
                # time.sleep(0.2)
                # driver.find_element_by_id("mi0,26").click()
                # time.sleep(0.4)

                # try:
                    # target =driver.find_element_by_xpath("//input[@value='查  詢']").click()
                    # # driver.execute_script("arguments[0].scrollIntoView();", target)
                    # # target.click()
                    # time.sleep(0.5)
                    # effDateOption=driver.find_element_by_id("format")
                    # effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
                    # for date in effDate:
                        # if "含保費" in date:
                            # driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            # time.sleep(0.2)
                    # time.sleep(0.8)
                    # driver.find_element_by_name("Submit722").click()
                # except:
                    # report.append("被保名冊下載失敗-證號:{}".format(uNo))
                # #勞退下載
                # try:
                    # time.sleep(0.3)
                    # driver.find_element_by_id("mi0,24").click()
                    # time.sleep(0.2)
                    # driver.find_element_by_id("mi0,34").click()
                    # time.sleep(0.4)
                    # driver.find_element_by_name("Submit").click()
                    # time.sleep(0.4)
                    # effDateOption=driver.find_element_by_id("hiddenidn")
                    # effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
                    # for date in effDate:
                        # if "全部顯示" in date:
                            # driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            # time.sleep(0.2)
                    # driver.find_element_by_xpath("//input[@value='TXT下載']").click()
                    # time.sleep(0.2)
                    # driver.back()
                
                    # effDateOption=driver.find_element_by_id("accountType")
                    # effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
                    # for date in effDate:
                        # if "個人" in date:
                            # driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            # time.sleep(0.2)
                    # driver.find_element_by_name("Submit").click()
                    # time.sleep(0.4)
                    # effDateOption=driver.find_element_by_id("hiddenidn")
                    # effDate=[i.strip() for i in effDateOption.text.split("\n")][1:]
                    # for date in effDate:
                        # if "全部顯示" in date:
                            # driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            # time.sleep(0.2)
                    # driver.find_element_by_xpath("//input[@value='TXT下載']").click()
                    # time.sleep(0.2)
                # except:
                    # report.append("勞退名冊下載失敗-證號:{}".format(uNo))
                # #############################################################        
                
#                 report.append("下載成功-證號:{}".format(uNo))
                try:
                    driver.find_element_by_id("mi0,53").click()
                except:
                    driver.find_element_by_id("mi0,52").click()
                time.sleep(1)

        
            except Exception as e:
                time.sleep(0.5)
                report.append("下載失敗-證號:{},加退調保全類,錯誤訊息:{},錯誤位置:{}".format(uNo,e,sys.exc_info()[2].tb_lineno))
                try:
                    driver.find_element_by_id("mi0,53").click()
                except:
                    driver.find_element_by_id("mi0,52").click()
                time.sleep(1)
        #########################################################更名區
        renamefile(path2)
        
        #########################################################
        
        driver.quit()
        e=time.time()
        costTime=round(e-b)
        report.append("\n花費時間:{}秒\n共處理{}筆資料".format(costTime,len(pathList)))
        print("\n".join(report))
        
        mailTo('Report',
               mails,
               web+"\n"+"\n".join(report))

    except Exception as e:
        if errorMsg=="":
            errorMsg=str(e)+"\n證號:{}\t位置:{}".format(unoNumber,excelFilePath)+'\nPlease verify website connection \n if no problem in website find andy.yuan@wowprime.com \n 出錯位置:'+str(sys.exc_info()[2].tb_lineno)
        
        print('出錯了RRRRRRRRRRRR'+str(e)+'\n')
        #driver.quit()
        
        mailTo('ReportError',mails,errorMsg)
        
def selectPath():
    path_ = askdirectory()
    path.set(path_)
def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)
with open('defaultIO.txt') as f:
    IO=f.read()
defaultLoadDir=IO.split("\n")[0].split("=")[1].replace("\\","/")
defaultSaveDir=IO.split("\n")[1].split("=")[1].replace("\\","/")
root = Tk()
path = StringVar()
Label(root,text = "讀取資料夾:").grid(row = 0, column = 0)
e1=Entry(root, textvariable = path)
e1.insert(0, defaultLoadDir)
e1.grid(row = 0, column = 1)
Button(root, text = "讀取資料夾", command = selectPath).grid(row = 0, column = 2)

path2 = StringVar()
Label(root,text = "存放資料夾:").grid(row = 1, column = 0)
e3=Entry(root, textvariable = path2)
e3.insert(0, defaultSaveDir)
e3.grid(row = 1, column = 1)

Button(root, text = "存放資料夾", command = selectPath2).grid(row = 1, column = 2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column = 0)
e2 = Entry(root)
e2.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='加退調保開始', command=doAddPopChangeAssure).grid(row=3, column=1, pady=4)
root.mainloop()

D:/保險申報檔案/test\11801_調1二合一.xls
D:/保險申報檔案/test
D:/保險申報檔案/test\1070210\11801_調2.xls
1070210
D:/保險申報檔案/test\1070210\11801_退2二合一.xls
1070210
出錯了RRRRRRRRRRRRMessage: chrome not reachable
  (Session info: chrome=64.0.3282.186)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 6.1.7601 SP1 x86_64)




# 下載勞健保清冊

In [2]:
from tkinter import Tk, StringVar, Label, Entry, Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait  # 直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC  # 判斷某物件存在，直到時間結束
import os
import sys
import time
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage

report = []
changDict = {"4": "加保", "2": "退保", "3": "調保"}
web = 'https://edesk.bli.gov.tw/cpa/'

with open('Email.txt') as f:
    mails = f.read().split('\n')
while "" in mails:
    mails.remove("")
mails = set(mails)


def mailTo(title, mailAdds, message, whoSend='AutoDownloadList'):
    msg = MIMEMultipart()
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text = msg.as_string()
    # print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1', 25)
    s.sendmail(sender, mailAdds, text)
    s.quit()


def doAddPopChangeAssure():
    path2 = e3.get()
    unoNumber = 0
    excelFilePath = ""

    #     path=r"D:\保險申報檔案\存放區"
    def renamefile(path):
        for pathTuple in os.walk(path):
            for file_ in pathTuple[2]:
                try:
                    if 'txt' in file_ and 'crdownload' not in file_:
                        name = []
                        uno = ""
                        savepath = ""
                        with open(os.path.join(pathTuple[0], file_), "r") as f:
                            datas = f.readline().split(",")
                            if '勞保名冊' in file_:
                                uno = mapAlias[datas[0] + datas[1].upper()]
                                date = datas[6]
                                style = "勞保"
                                savepath = pathTuple[0] + "\\勞保"
                            elif '勞退' in file_ and '雇主' in file_:
                                uno = mapAlias[datas[0][1:] + datas[1].upper()]
                                date = datas[2]
                                style = "勞退"
                                savepath = pathTuple[0] + "\\勞退"
                            if '勞退' in file_ and '個人' in file_:
                                uno = mapAlias[datas[0][1:] + datas[1].upper()]
                                date = datas[2]
                                style = "勞退自提"
                                savepath = pathTuple[0] + "\\勞退自提"

                            if savepath and not os.path.exists(savepath):
                                os.makedirs(savepath)
                        try:
                            print(file_)
                            if uno:
                                os.rename(os.path.join(pathTuple[0], file_),
                                          os.path.join(savepath, uno + style + "_" + date + ".txt"))
                        except Exception as e:
                            print(e)
                            print(os.path.join(path, file_) + "\t已有變更檔名的檔案")
                            os.remove(os.path.join(pathTuple[0], file_))
                except Exception as e:
                    report.append("更名失敗>>{},錯誤:{},出錯位置:{}".format(os.path.join(pathTuple[0], file_), e,
                                                                  sys.exc_info()[2].tb_lineno))

    try:
        report = []
        b = time.time()
        errorMsg = ""
        pinNum = e2.get()  # pin號碼
        chrome_options = Options()
        # crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔
        chrome_options.add_extension('extension_0_0_1_3.crx')

        chrome_options.add_experimental_option("prefs", {
            "download.default_directory": e3.get(),
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        })

        # webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
        # 套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
        driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
        driver.get(web)

        dataDir = e1.get()

        pathList = [i[0] + "\\" + j for i in os.walk(dataDir) for j in i[2] if 'xlsx' in j and "$" not in j]
        print(pathList)
        for excelFilePath in pathList:
            data = xlrd.open_workbook(excelFilePath)
            table = data.sheet_by_index(0)
            for i in range(10):
                try:
                    for text in table.col_values(i)[:7]:
                        try:
                            if '店別' == text:
                                print('xxx')
                                mapAlias = table.col_values(i)
                                aliasNumberList = [str(int(no)) for no in table.col_values(i) if
                                                   no != "" and (type(no) == float or type(no) == int)]
                            #                                 print(aliasNumberList)
                            if '保險證字號' in text:
                                mapUno = table.col_values(i)
                                #                                 print(sheet.col_values(i))
                                print(text)
                                unoNumberList = [no for no in table.col_values(i) if
                                                 no != "" and no[0] == "0" and len(no) == 9]
                                #                                 print(unoNumberList)
                                break
                            else:
                                continue
                        except:
                            pass
                except:
                    pass

        mapAlias = {uno.upper(): str(int(alias)) for uno, alias in zip(mapUno, mapAlias) if
                    alias != "" and (type(alias) == float or type(alias) == int)}

        print(mapAlias)

        for unoNumber in unoNumberList:
            try:
                # 進入勞保局
                WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CLASS_NAME, 'b-close')))
                time.sleep(0.4)
                #     alert=driver.switch_to_alert()
                #     alert.dismiss()
                # 關閉阻擋頁
                successful = 0
                sumN = 0
                while successful == 0:
                    try:
                        sumN += 1
                        if sumN > 20:
                            break
                        driver.find_element_by_class_name("b-close").click()
                        successful = 1
                    except:
                        time.sleep(0.3)
                time.sleep(0.2)
                # 輸入pin碼(自然人憑證)
                pinInput = driver.find_element_by_name("pin")
                pinInput.clear()
                pinInput.send_keys(pinNum)  # 憑證pin碼
                # 輸入保險證號
                unoInput = driver.find_element_by_name("uno")
                unoInput.clear()
                unoInput.send_keys(unoNumber)
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.NAME, 'Submit')))
                time.sleep(0.5)
                # 確認進入
                successful = 0
                sumN = 0
                while successful == 0:
                    try:
                        sumN += 1
                        if sumN > 100:
                            break
                        driver.find_element_by_name('Submit').click()
                        successful = 1
                    except:
                        time.sleep(0.3)
                time.sleep(2)
                try:
                    WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CLASS_NAME, 'b-close')))
                    time.sleep(0.4)
                    # driver.find_element_by_class_name("b-close").click()
                    successful = 0
                    sumN = 0
                    while successful == 0:
                        try:
                            sumN += 1
                            if sumN > 20:
                                break
                            driver.find_element_by_class_name("b-close").click()
                            successful = 1
                        except:
                            time.sleep(0.3)
                    time.sleep(0.4)
                except:
                    report.append("登入失敗-\t證號:{}".
                                  format(unoNumber))
                    time.sleep(0.3)
                    alert = driver.switch_to_alert()
                    ###############################################判斷密碼錯誤、權限錯誤問題
                    if '讀卡機' in alert.text:
                        errorMsg = "讀卡機沒插卡"
                    elif '權限' in alert.text:
                        errorMsg = "你沒權限跑{}".format(unoNumber)
                    elif '密碼' in alert.text:
                        errorMsg = "密碼錯誤>>你所輸入的密碼是:{}".format(pinNum)

                    if errorMsg != "":
                        mailTo('ReportError', mails, errorMsg)
                        driver.quit()
                        root.destroy()
                        sys.exit("因故中斷:" + errorMsg)
                    ###############################################
                    alert.accept()
                    continue
                # #########################################################被保名冊下載
                driver.find_element_by_id("mi0,24").click()
                time.sleep(0.2)
                driver.find_element_by_id("mi0,26").click()
                time.sleep(0.4)

                try:
                    target = driver.find_element_by_xpath("//input[@value='查  詢']").click()
                    # driver.execute_script("arguments[0].scrollIntoView();", target)
                    # target.click()
                    time.sleep(0.5)
                    effDateOption = driver.find_element_by_id("format")
                    effDate = [i.strip() for i in effDateOption.text.split("\n")][1:]
                    for date in effDate:
                        if "含保費" in date:
                            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            time.sleep(0.2)
                    time.sleep(0.8)
                    driver.find_element_by_name("Submit722").click()
                except:
                    report.append("被保名冊下載失敗-證號:{},錯誤位置:{}".format(unoNumber,sys.exc_info()[2].tb_lineno))
                # 勞退下載
                try:
                    time.sleep(0.3)
                    driver.find_element_by_id("mi0,24").click()
                    time.sleep(0.2)
                    driver.find_element_by_id("mi0,34").click()
                    time.sleep(0.4)
                    driver.find_element_by_name("Submit").click()
                    time.sleep(0.4)
                    effDateOption = driver.find_element_by_id("hiddenidn")
                    effDate = [i.strip() for i in effDateOption.text.split("\n")][1:]
                    for date in effDate:
                        if "全部顯示" in date:
                            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            time.sleep(0.2)
                    driver.find_element_by_xpath("//input[@value='TXT下載']").click()
                    time.sleep(0.2)
                    driver.back()

                    effDateOption = driver.find_element_by_id("accountType")
                    effDate = [i.strip() for i in effDateOption.text.split("\n")][1:]
                    for date in effDate:
                        if "個人" in date:
                            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            time.sleep(0.2)
                    driver.find_element_by_name("Submit").click()
                    time.sleep(0.4)
                    effDateOption = driver.find_element_by_id("hiddenidn")
                    effDate = [i.strip() for i in effDateOption.text.split("\n")][1:]
                    for date in effDate:
                        if "全部顯示" in date:
                            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            time.sleep(0.2)
                    driver.find_element_by_xpath("//input[@value='TXT下載']").click()
                    time.sleep(0.2)
                except:
                    report.append("勞退名冊下載失敗-證號:{},錯誤位置{}".format(unoNumber,sys.exc_info()[2].tb_lineno))
                # #############################################################

                #                 report.append("下載成功-證號:{}".format(uNo))
                try:
                    driver.find_element_by_id("mi0,53").click()
                except:
                    driver.find_element_by_id("mi0,52").click()
                time.sleep(1)


            except Exception as e:
                time.sleep(0.5)
                report.append("勞保清冊下載失敗-證號:{},錯誤訊息:{},錯誤位置:{}".format(unoNumber, e, sys.exc_info()[2].tb_lineno))
                driver.get(web)
                # try:
                #     driver.find_element_by_id("mi0,53").click()
                # except:
                #     driver.find_element_by_id("mi0,52").click()
                time.sleep(1)
        #########################################################更名區
        renamefile(path2)

        #########################################################

        driver.quit()
        e = time.time()
        costTime = round(e - b)
        report.append("勞保清冊下載完畢\n花費時間:{}秒\n共處理{}筆證號".format(costTime, len(unoNumberList)))
        print("\n".join(report))

        mailTo('ReportDownList',
               mails,
               web + "\n" + "\n".join(report))

    except Exception as e:
        if errorMsg == "":
            errorMsg = str(e) + "\n證號:{}".format(
                unoNumber) + '\nPlease verify website connection \n if no problem in website find andy.yuan@wowprime.com \n 出錯位置:' + str(
                sys.exc_info()[2].tb_lineno)

        print('出錯了RRRRRRRRRRRR' + str(e) + '\n')
        # driver.quit()

        mailTo('ReportError', mails, errorMsg)


def selectPath():
    path_ = askdirectory()
    path.set(path_)


def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)


with open('defaultIO.txt') as f:
    IO = f.read()
defaultLoadDir = IO.split("\n")[0].split("=")[1].replace("\\", "/")
defaultSaveDir = IO.split("\n")[1].split("=")[1].replace("\\", "/")
root = Tk()
path = StringVar()
Label(root, text="讀取資料夾:").grid(row=0, column=0)
e1 = Entry(root, textvariable=path)
e1.insert(0, defaultLoadDir)
e1.grid(row=0, column=1)
Button(root, text="讀取資料夾", command=selectPath).grid(row=0, column=2)

path2 = StringVar()
Label(root, text="存放資料夾:").grid(row=1, column=0)
e3 = Entry(root, textvariable=path2)
e3.insert(0, defaultSaveDir)
e3.grid(row=1, column=1)

Button(root, text="存放資料夾", command=selectPath2).grid(row=1, column=2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column=0)
e2 = Entry(root)
e2.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='下載勞保清冊開始', command=doAddPopChangeAssure).grid(row=3, column=1, pady=4)
root.mainloop()

['D:/保險申報檔案/test_for_下載勞健保清冊\\各單位勞.健.勞退證號.xlsx']
xxx
勞保
保險證字號
{'05292898H': '10100', '05606625W': '10202', '05636060B': '10203', '05896411F': '10204', '05648576B': '10205', '05661119X': '10206', '05663470Y': '10207', '05675971W': '10208', '05678340X': '10209', '05693173S': '10210', '05700386X': '10211', '05778079T': '10215', '05802967G': '10216', '05801400Y': '10217', '05831794A': '10220', '05869013B': '10221', '05878028S': '10222', '05859019B': '10225', '05964411X': '10227', '05740415Y': '10301', '05429455G': '10302', '05918782Y': '10304', '05459948B': '10306', '05369674F': '10307', '05930764T': '10308', '05423337Y': '10309', '05417285S': '10311', '05902201A': '10313', '05431641A': '10314', '05943066S': '10316', '05789313K': '10317', '05847565X': '10319', '05591675S': '10401', '05749323W': '10411', '05752685H': '10413', '05761640K': '10416', '05912254G': '10419', '05783044H': '10420', '05789474S': '10422', '05813420K': '10424', '05810555T': '10425', '05816443A': '10427', '05812696Y': 

SystemExit: 因故中斷:讀卡機沒插卡

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [150]:
renamefile(path)

["更名失敗>>D:\\保險申報檔案\\存放區\\05952199-10701勞保名冊-20180312.txt,錯誤:'5952199X',出錯位置:11", "更名失敗>>D:\\保險申報檔案\\存放區\\05952202-10701勞保名冊-20180312.txt,錯誤:'5952202F',出錯位置:11", "更名失敗>>D:\\保險申報檔案\\存放區\\05952199-10701勞保名冊-20180312.txt,錯誤:'5952199X',出錯位置:11", "更名失敗>>D:\\保險申報檔案\\存放區\\05952202-10701勞保名冊-20180312.txt,錯誤:'5952202F',出錯位置:11", "更名失敗>>D:\\保險申報檔案\\存放區\\05952199-10701勞保名冊-20180312.txt,錯誤:'5952199X',出錯位置:11"]
["更名失敗>>D:\\保險申報檔案\\存放區\\05952199-10701勞保名冊-20180312.txt,錯誤:'5952199X',出錯位置:11", "更名失敗>>D:\\保險申報檔案\\存放區\\05952202-10701勞保名冊-20180312.txt,錯誤:'5952202F',出錯位置:11", "更名失敗>>D:\\保險申報檔案\\存放區\\05952199-10701勞保名冊-20180312.txt,錯誤:'5952199X',出錯位置:11", "更名失敗>>D:\\保險申報檔案\\存放區\\05952202-10701勞保名冊-20180312.txt,錯誤:'5952202F',出錯位置:11", "更名失敗>>D:\\保險申報檔案\\存放區\\05952199-10701勞保名冊-20180312.txt,錯誤:'5952199X',出錯位置:11", "更名失敗>>D:\\保險申報檔案\\存放區\\05952202-10701勞保名冊-20180312.txt,錯誤:'5952202F',出錯位置:11"]


In [149]:
path=r"D:\保險申報檔案\存放區"
def renamefile(path):
#         pathList2=[i[0]+"/"+j for i in os.walk(path) for j in i[2]]
    for pathTuple in os.walk(path):
        for file_ in pathTuple[2]:
            try:
                if 'txt' in file_ and 'crdownload' not in file_:
                    name=[]
                    with open(os.path.join(pathTuple[0], file_),"r") as f:
                        datas=f.readline().split(",")
                        if '勞保名冊' in file_:
                            uno=mapAlias[datas[0][1:]+datas[1]]
                            date=datas[6]
                            style="勞保"
                        elif '勞退' in file_:
                            uno=mapAlias[datas[0]+datas[1]]
                            date=datas[2]
                            style="勞保"

                    try:
                        os.rename(os.path.join(pathTuple[0],file_),os.path.join(pathTuple[0],uno+"_"+style+"_"+date+".txt"))
                    except:
    #                         print(os.path.join(path, file_)+"\t已有變更檔名的檔案")
                        os.remove(os.path.join(pathTuple[0],file_))
            except Exception as e:
                report.append("更名失敗>>{},錯誤:{},出錯位置:{}".format(os.path.join(pathTuple[0],file_),e,sys.exc_info()[2].tb_lineno))

In [11]:
import os
import xlrd
dataDir=r"D:\保險申報檔案\test_for_下載勞健保清冊"
pathList=[i[0]+"\\"+j for i in os.walk(dataDir) for j in i[2] if 'xlsx' in j and "$" not in j]
print(pathList)
for excelFilePath in pathList:
    data = xlrd.open_workbook(excelFilePath)
    table=data.sheet_by_index(0)
    for i in range(10):
        try:
            for text in table.col_values(i)[:7]:
                try:
                    if '店別' == text:
#                         print('xxx')
#                         print(i)
                        aliasNumberList=[str(int(no)) for no in table.col_values(i) if no!="" and (type(no)==float or type(no)==int)]
#                         print('yyy')
#                         print(aliasNumberList)
                    if '保險證字號' in text:
#                         print(sheet.col_values(i))
                        print(text)
                        unoNumberList=[no for no in table.col_values(i) if no!="" and no[0]=="0" and len(no)==9]
#                         print(unoNumberList)
                        break
                    else:
                        continue
                except:
                    pass
        except:
            pass

mapAlias={uno:alias for uno,alias in zip(unoNumberList,aliasNumberList)}
print(mapAlias)

['D:\\保險申報檔案\\test_for_下載勞健保清冊\\各單位勞.健.勞退證號.xlsx']
勞保
保險證字號
{'05292898H': '10100', '05606625W': '10202', '05636060B': '10203', '05896411F': '10204', '05648576B': '10205', '05661119X': '10206', '05663470Y': '10207', '05675971W': '10208', '05678340X': '10209', '05693173S': '10210', '05700386X': '10211', '05778079T': '10215', '05802967G': '10216', '05801400Y': '10217', '05831794A': '10220', '05869013B': '10221', '05878028S': '10222', '05859019B': '10225', '05964411X': '10227', '05740415Y': '10301', '05429455G': '10302', '05918782Y': '10304', '05459948B': '10306', '05369674F': '10307', '05930764T': '10308', '05423337Y': '10309', '05417285S': '10311', '05902201A': '10313', '05431641A': '10314', '05943066S': '10316', '05789313k': '10317', '05847565X': '10319', '05591675S': '10401', '05749323W': '10411', '05752685H': '10413', '05761640K': '10416', '05912254G': '10419', '05783044H': '10420', '05789474S': '10422', '05813420K': '10424', '05810555T': '10425', '05816443A': '10427', '05812696Y': '1

In [42]:
path=r"D:\保險申報檔案\存放區"
report=[]
for pathTuple in os.walk(path):
    for file_ in pathTuple[2]:
        try:
            if 'txt' in file_ and 'crdownload' not in file_:
                name=[]
                uno=""
                savepath=""
                with open(os.path.join(pathTuple[0], file_),"r") as f:
                    datas=f.readline().split(",")
                    if '勞保名冊' in file_:
                        uno=mapAlias[datas[0]+datas[1]]
                        date=datas[6]
                        style="勞保"
                        savepath=pathTuple[0]+"\\勞保"
                    elif '勞退' in file_ and '雇主' in file_:
                        uno=mapAlias[datas[0][1:]+datas[1]]
                        date=datas[2]
                        style="勞退"
                        savepath=pathTuple[0]+"\\勞退"
                    if '勞退' in file_ and '個人' in file_:
                        uno=mapAlias[datas[0][1:]+datas[1]]
                        date=datas[2]
                        style="勞退自提"
                        savepath=pathTuple[0]+"\\勞退自提"
                    
                    if savepath and not os.path.exists(savepath):
                        os.makedirs(savepath)
                try:
                    print(file_)
                    if uno:
                        os.rename(os.path.join(pathTuple[0],file_),
                                  os.path.join(savepath,uno+"_"+style+"_"+date+".txt"))
                except Exception as e:
                    print(e)
                    print(os.path.join(path, file_)+"\t已有變更檔名的檔案")
                    os.remove(os.path.join(pathTuple[0],file_))
        except Exception as e:
            report.append("更名失敗>>{},錯誤:{},出錯位置:{}".format(os.path.join(pathTuple[0],file_),e,
                                                          sys.exc_info()[2].tb_lineno))
    

10705_勞保_10701.txt
10705_勞退_10701.txt
10705_勞退自提_10701.txt
[]


In [39]:
savepath

'D:\\保險申報檔案\\存放區\\勞退'

In [40]:
if savepath and not os.path.exists(savepath):
        os.makedirs(savedir)

NameError: name 'savedir' is not defined

In [ ]:
from tkinter import Tk, StringVar, Label, Entry, Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait  # 直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC  # 判斷某物件存在，直到時間結束
import os
import sys
import time
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage

report = []
changDict = {"4": "加保", "2": "退保", "3": "調保"}
web = 'https://edesk.bli.gov.tw/cpa/'

with open('Email.txt') as f:
    mails = f.read().split('\n')
while "" in mails:
    mails.remove("")
mails = set(mails)


def mailTo(title, mailAdds, message, whoSend='AutoDownloadList'):
    msg = MIMEMultipart()
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text = msg.as_string()
    # print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1', 25)
    s.sendmail(sender, mailAdds, text)
    s.quit()


def doAddPopChangeAssure():
    path2 = e3.get()
    unoNumber = 0
    excelFilePath = ""

    #     path=r"D:\保險申報檔案\存放區"
    def renamefile(path):
        for pathTuple in os.walk(path):
            for file_ in pathTuple[2]:
                try:
                    if 'txt' in file_ and 'crdownload' not in file_:
                        name = []
                        uno = ""
                        savepath = ""
                        with open(os.path.join(pathTuple[0], file_), "r") as f:
                            datas = f.readline().split(",")
                            if '勞保名冊' in file_:
                                uno = mapAlias[datas[0] + datas[1].upper()]
                                date = datas[6]
                                style = "勞保"
                                savepath = pathTuple[0] + "\\勞保"
                            elif '勞退' in file_ and '雇主' in file_:
                                uno = mapAlias[datas[0][1:] + datas[1].upper()]
                                date = datas[2]
                                style = "勞退"
                                savepath = pathTuple[0] + "\\勞退"
                            if '勞退' in file_ and '個人' in file_:
                                uno = mapAlias[datas[0][1:] + datas[1].upper()]
                                date = datas[2]
                                style = "勞退自提"
                                savepath = pathTuple[0] + "\\勞退自提"

                            if savepath and not os.path.exists(savepath):
                                os.makedirs(savepath)
                        try:
                            print(file_)
                            if uno:
                                os.rename(os.path.join(pathTuple[0], file_),
                                          os.path.join(savepath, uno + style + "_" + date + ".txt"))
                        except Exception as e:
                            print(e)
                            print(os.path.join(path, file_) + "\t已有變更檔名的檔案")
                            os.remove(os.path.join(pathTuple[0], file_))
                except Exception as e:
                    report.append("更名失敗>>{},錯誤:{},出錯位置:{}".format(os.path.join(pathTuple[0], file_), e,
                                                                  sys.exc_info()[2].tb_lineno))

    try:
        report = []
        b = time.time()
        errorMsg = ""
        pinNum = e2.get()  # pin號碼
        chrome_options = Options()
        # crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔
        chrome_options.add_extension('extension_0_0_1_3.crx')

        chrome_options.add_experimental_option("prefs", {
            "download.default_directory": e3.get(),
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        })

        # webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
        # 套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
        driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
        driver.get(web)

        dataDir = e1.get()
        
        pathList = [i[0] + "\\" + j for i in os.walk(dataDir) for j in i[2] if 'xlsx' in j and "$" not in j]
        print(pathList)
        for excelFilePath in pathList:
            data = xlrd.open_workbook(excelFilePath)
            table = data.sheet_by_index(0)
            for i in range(10):
                try:
                    for text in table.col_values(i)[:7]:
                        try:
                            if '店別' == text:
                                print('xxx')
                                mapAlias = table.col_values(i)
                                aliasNumberList = [str(int(no)) for no in table.col_values(i) if
                                                   no != "" and (type(no) == float or type(no) == int)]
                            #                                 print(aliasNumberList)
                            if '保險證字號' in text:
                                mapUno = table.col_values(i)
                                #                                 print(sheet.col_values(i))
                                print(text)
                                unoNumberList = [no for no in table.col_values(i) if
                                                 no != "" and no[0] == "0" and len(no) == 9]
                                #                                 print(unoNumberList)
                                break
                            else:
                                continue
                        except:
                            pass
                except:
                    pass

        mapAlias = {uno.upper(): str(int(alias)) for uno, alias in zip(mapUno, mapAlias) if
                    alias != "" and (type(alias) == float or type(alias) == int)}

        print(mapAlias)

        for unoNumber in unoNumberList:
            try:
                # 進入勞保局
                WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CLASS_NAME, 'b-close')))
                time.sleep(0.4)
                #     alert=driver.switch_to_alert()
                #     alert.dismiss()
                # 關閉阻擋頁
                successful = 0
                sumN = 0
                while successful == 0:
                    try:
                        sumN += 1
                        if sumN > 20:
                            break
                        driver.find_element_by_class_name("b-close").click()
                        successful = 1
                    except:
                        time.sleep(0.3)
                time.sleep(0.2)
                # 輸入pin碼(自然人憑證)
                pinInput = driver.find_element_by_name("pin")
                pinInput.clear()
                pinInput.send_keys(pinNum)  # 憑證pin碼
                # 輸入保險證號
                unoInput = driver.find_element_by_name("uno")
                unoInput.clear()
                unoInput.send_keys(unoNumber)
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.NAME, 'Submit')))
                time.sleep(0.5)
                # 確認進入
                successful = 0
                sumN = 0
                while successful == 0:
                    try:
                        sumN += 1
                        if sumN > 100:
                            break
                        driver.find_element_by_name('Submit').click()
                        successful = 1
                    except:
                        time.sleep(0.3)
                time.sleep(2)
                try:
                    WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CLASS_NAME, 'b-close')))
                    time.sleep(0.4)
                    # driver.find_element_by_class_name("b-close").click()
                    successful = 0
                    sumN = 0
                    while successful == 0:
                        try:
                            sumN += 1
                            if sumN > 20:
                                break
                            driver.find_element_by_class_name("b-close").click()
                            successful = 1
                        except:
                            time.sleep(0.3)
                    time.sleep(0.4)
                except:
                    report.append("登入失敗-\t證號:{}".
                                  format(unoNumber))
                    time.sleep(0.3)
                    alert = driver.switch_to_alert()
                    ###############################################判斷密碼錯誤、權限錯誤問題
                    if '讀卡機' in alert.text:
                        errorMsg = "讀卡機沒插卡"
                    elif '權限' in alert.text:
                        errorMsg = "你沒權限跑{}".format(unoNumber)
                    elif '密碼' in alert.text:
                        errorMsg = "密碼錯誤>>你所輸入的密碼是:{}".format(pinNum)

                    if errorMsg != "":
                        mailTo('ReportError', mails, errorMsg)
                        driver.quit()
                        root.destroy()
                        sys.exit("因故中斷:" + errorMsg)
                    ###############################################
                    alert.accept()
                    continue
                # #########################################################被保名冊下載
                driver.find_element_by_id("mi0,24").click()
                time.sleep(0.2)
                driver.find_element_by_id("mi0,26").click()
                time.sleep(0.4)

                try:
                    target = driver.find_element_by_xpath("//input[@value='查  詢']").click()
                    # driver.execute_script("arguments[0].scrollIntoView();", target)
                    # target.click()
                    time.sleep(0.5)
                    effDateOption = driver.find_element_by_id("format")
                    effDate = [i.strip() for i in effDateOption.text.split("\n")][1:]
                    for date in effDate:
                        if "含保費" in date:
                            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            time.sleep(0.2)
                    time.sleep(0.8)
                    driver.find_element_by_name("Submit722").click()
                except:
                    report.append("被保名冊下載失敗-證號:{},錯誤位置:{}".format(unoNumber,sys.exc_info()[2].tb_lineno))
                # 勞退下載
                try:
                    time.sleep(0.3)
                    driver.find_element_by_id("mi0,24").click()
                    time.sleep(0.2)
                    driver.find_element_by_id("mi0,34").click()
                    time.sleep(0.4)
                    driver.find_element_by_name("Submit").click()
                    time.sleep(0.4)
                    effDateOption = driver.find_element_by_id("hiddenidn")
                    effDate = [i.strip() for i in effDateOption.text.split("\n")][1:]
                    for date in effDate:
                        if "全部顯示" in date:
                            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            time.sleep(0.2)
                    driver.find_element_by_xpath("//input[@value='TXT下載']").click()
                    time.sleep(0.2)
                    driver.back()

                    effDateOption = driver.find_element_by_id("accountType")
                    effDate = [i.strip() for i in effDateOption.text.split("\n")][1:]
                    for date in effDate:
                        if "個人" in date:
                            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            time.sleep(0.2)
                    driver.find_element_by_name("Submit").click()
                    time.sleep(0.4)
                    effDateOption = driver.find_element_by_id("hiddenidn")
                    effDate = [i.strip() for i in effDateOption.text.split("\n")][1:]
                    for date in effDate:
                        if "全部顯示" in date:
                            driver.find_element_by_xpath('//option[text()="{}"]'.format(date)).click()
                            time.sleep(0.2)
                    driver.find_element_by_xpath("//input[@value='TXT下載']").click()
                    time.sleep(0.2)
                except:
                    report.append("勞退名冊下載失敗-證號:{},錯誤位置{}".format(unoNumber,sys.exc_info()[2].tb_lineno))
                # #############################################################

                #                 report.append("下載成功-證號:{}".format(uNo))
                try:
                    driver.find_element_by_id("mi0,53").click()
                except:
                    driver.find_element_by_id("mi0,52").click()
                time.sleep(1)


            except Exception as e:
                time.sleep(0.5)
                report.append("勞保清冊下載失敗-證號:{},錯誤訊息:{},錯誤位置:{}".format(unoNumber, e, sys.exc_info()[2].tb_lineno))
                driver.get(web)
                # try:
                #     driver.find_element_by_id("mi0,53").click()
                # except:
                #     driver.find_element_by_id("mi0,52").click()
                time.sleep(1)
        #########################################################更名區
        renamefile(path2)

        #########################################################

        driver.quit()
        e = time.time()
        costTime = round(e - b)
        report.append("勞保清冊下載完畢\n花費時間:{}秒\n共處理{}筆證號".format(costTime, len(unoNumberList)))
        print("\n".join(report))

        mailTo('ReportDownList',
               mails,
               web + "\n" + "\n".join(report))

    except Exception as e:
        if errorMsg == "":
            errorMsg = str(e) + "\n證號:{}".format(
                unoNumber) + '\nPlease verify website connection \n if no problem in website find andy.yuan@wowprime.com \n 出錯位置:' + str(
                sys.exc_info()[2].tb_lineno)

        print('出錯了RRRRRRRRRRRR' + str(e) + '\n')
        # driver.quit()

        mailTo('ReportError', mails, errorMsg)


def selectPath():
    path_ = askdirectory()
    path.set(path_)


def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)


with open('defaultIO.txt') as f:
    IO = f.read()
defaultLoadDir = IO.split("\n")[0].split("=")[1].replace("\\", "/")
defaultSaveDir = IO.split("\n")[1].split("=")[1].replace("\\", "/")
root = Tk()
path = StringVar()
Label(root, text="讀取資料夾:").grid(row=0, column=0)
e1 = Entry(root, textvariable=path)
e1.insert(0, defaultLoadDir)
e1.grid(row=0, column=1)
Button(root, text="讀取資料夾", command=selectPath).grid(row=0, column=2)

path2 = StringVar()
Label(root, text="存放資料夾:").grid(row=1, column=0)
e3 = Entry(root, textvariable=path2)
e3.insert(0, defaultSaveDir)
e3.grid(row=1, column=1)

Button(root, text="存放資料夾", command=selectPath2).grid(row=1, column=2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column=0)
e2 = Entry(root)
e2.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='下載勞保清冊開始', command=doAddPopChangeAssure).grid(row=3, column=1, pady=4)
root.mainloop()

# 健保下載!!!!!!!!!!!!!!

In [1]:
from tkinter import Tk, StringVar, Label, Entry, Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait  # 直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC  # 判斷某物件存在，直到時間結束
import os
import sys
import time
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage

In [2]:
report = []
changDict = {"4": "加保", "2": "退保", "3": "調保"}
web = 'https://eservice.nhi.gov.tw/nhiweb1/system/mLoginCa.aspx'

with open('Email.txt') as f:
    mails = f.read().split('\n')
while "" in mails:
    mails.remove("")
mails = set(mails)


def mailTo(title, mailAdds, message, whoSend='AutoDownloadList'):
    msg = MIMEMultipart()
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text = msg.as_string()
    # print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1', 25)
    s.sendmail(sender, mailAdds, text)
    s.quit()



In [13]:
report = []
b = time.time()
errorMsg = ""
pinNum = "780311"  # pin號碼
chrome_options = Options()
# crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔
# chrome_options.add_extension('extension_0_0_1_3.crx')

chrome_options.add_experimental_option("prefs", {
    "download.default_directory":r"D:\保險申報檔案\存放區",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
# 套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
driver.get(web)

dataDir = r"D:\保險申報檔案\test_for_下載勞健保清冊"
inputPW=driver.find_element_by_id("USR_PASSWORD1")
inputPW.clear()
inputPW.send_keys(pinNum)
driver.find_element_by_name("btnSubmit").click()

In [14]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'UNIT_ID')))
time.sleep(1)

In [24]:
pathList = [i[0] + "\\" + j for i in os.walk(dataDir) for j in i[2] if 'xlsx' in j and "$" not in j]

for excelFilePath in pathList:
    data = xlrd.open_workbook(excelFilePath)
    table = data.sheet_by_index(0)
    for i in range(10):
        try:
            for text in table.col_values(i)[:7]:
#                 print(text+"---")
                try:
                    if '店別' == text:
#                         print('xxx')
                        mapAliass = table.col_values(i)
                        aliasNumberList = [str(int(no)) for no in mapAliass if
                                           no != "" and (type(no) == float or type(no) == int)]
                    #                                 print(aliasNumberList)
                    if '健保投保單位代號'==text:
                        mapUno = table.col_values(i)
                        print(text)
                        unoNumberList = [str(int(no)) for no in mapUno if
                                         no != "" and type(no) == float]
                        #                                 print(unoNumberList)
                        break
                    else:
                        continue
                except:
                    pass
        except:
            pass

mapAlias = {str(int(alias)):str(int(uno))  for uno, alias in zip(mapAliass, mapUno) if
                                           uno != "" and (type(uno) == float or type(uno) == int)
                                           and alias!= "" and type(alias) == float}

print(mapAlias)

健保投保單位代號
{'145345099': '11801', '145345080': '11802'}


In [18]:
len(mapAliass)

298

In [9]:
num="145345080"
driver.find_element_by_id("UNIT_ID").click()
time.sleep(0.2)
driver.find_element_by_xpath('//option[@value="'+num+'"]').click()

In [10]:
driver.find_element_by_name("btnSubmit").click()
time.sleep(0.4)


In [12]:
driver.switch_to_frame(0)
driver.find_element_by_id("MenuMaker1_TreeView1t30").text

'各類明細表申請及下載'

In [402]:
driver.switch_to_default_content()
driver.switch_to_frame(0)

In [405]:
x

'各類明細表申請及下載'

In [294]:
n=0
while n<2:
    for num in mapAlias:
        driver.find_element_by_id("UNIT_ID").click()
        time.sleep(0.2)
        driver.find_element_by_xpath('//option[@value="'+num+'"]').click()
        driver.find_element_by_name("btnSubmit").click()
        time.sleep(0.4)
        driver.switch_to_frame(0)
        driver.find_element_by_id("MenuMaker1_TreeView1t31").click()
        if n ==0:
            driver.find_element_by_id("MenuMaker1_TreeView1t32").click()
            time.sleep(1)
            driver.switch_to_frame(0)
            driver.find_element_by_xpath('//option[@value="02"]').click()
            time.sleep(0.3)
            driver.find_element_by_id("ToolBar1_btnAdd").click()
            time.sleep(0.3)
            x=driver.switch_to_alert()
            time.sleep(0.2)
            x.accept()
        else:
            driver.find_element_by_id("MenuMaker1_TreeView1t33").click()
            time.sleep(1)
            driver.switch_to_frame(0)
            driver.find_element_by_xpath('//option[@value="02"]').click()
            time.sleep(0.3)
            driver.find_element_by_id("ToolBar1_btnQuery").click()
            time.sleep(0.3)
            driver.find_element_by_name("DataGrid1$ctl02$ctl00").click()
            time.sleep(0.3)
        driver.switch_to_default_content()
        driver.find_element_by_class_name("Logout").click()
        time.sleep(0.5)
    n+=1

In [278]:
driver.find_element_by_xpath('//option[@value="'+num+'"]').click()
driver.find_element_by_name("btnSubmit").click()

In [279]:
driver.switch_to_frame(0)

In [224]:
driver.find_element_by_id("MenuMaker1_TreeView1t31").click()
driver.find_element_by_id("MenuMaker1_TreeView1t32").click()
time.sleep(1)
driver.switch_to_frame(0)
driver.find_element_by_xpath('//option[@value="02"]').click()
driver.find_element_by_id("ToolBar1_btnAdd").click()
x=driver.switch_to_alert()
x.accept()
driver.switch_to_default_content()

In [255]:
driver.find_element_by_id("btnLogout").click()

In [252]:
driver.switch_to_default_content()

In [253]:
driver.page_source

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=9,IE=10,IE=11,IE=EDGE" /><title>\n\t多憑證網路承保作業\n</title><link rel="stylesheet" href="../Styles/Menu.css" type="text/css" /></head>\n<body style="margin: 0px; height: 919px;" id="MainBody">\n    <form method="post" action="Main.aspx?UNIT_NAME=%u7fa4%u54c1%u9910%u98f2%u80a1%u4efd%u6709%u9650%u516c%u53f8%u9177%u5fc5%u53f0%u5317%u6c40%u6d32%u5e97" id="form1">\n<div class="aspNetHidden">\n<input type="hidden" name="__VIEWSTATE" id="__VIEWSTATE" value="NEd1uZXhhGYfUCUTj+lSsS8eZ5IztmPIW9B3w20nsvqGJVx4wrgPzLQKG3DYrsJvXICDOakpgELSVKMPHnsUd64HRpxyk0e5LAE5Png2/+ZJKwfSBw6v8F06BBMO8zupJNvxKEGHn8OUDQfN84STV4YPc8b0FBB6mkJMc8QcpGC++DbZYObQ5NMLHkTRWB/Usz086C6RSo71Z6eF8fbzD6ogb/X5Jr38Os7etGrt8oJI5+akXGSA3zByOAlImX1vpzfKHvW

In [282]:
driver.find_element_by_id("MenuMaker1_TreeView1t31").click()
driver.find_element_by_id("MenuMaker1_TreeView1t33").click()
time.sleep(1)
driver.switch_to_frame(0)
driver.find_element_by_xpath('//option[@value="02"]').click()
time.sleep(0.3)
driver.find_element_by_id("ToolBar1_btnQuery").click()
time.sleep(0.3)
driver.find_element_by_name("DataGrid1$ctl02$ctl00").click()
time.sleep(0.3)

WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"Cannot find context with specified id"}
  (Session info: chrome=65.0.3325.181)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 6.1.7601 SP1 x86_64)


In [ ]:
driver.find_element_by_xpath('//option[@value="02"]').click()
driver.find_element_by_id("ToolBar1_btnAdd").click()
x=driver.switch_to_alert()
x.accept()
driver.switch_to_default_content()

# 一次申請+下載 健保清冊

In [ ]:
from tkinter import Tk, StringVar, Label, Entry, Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait  # 直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC  # 判斷某物件存在，直到時間結束
import os
import sys
import time
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage

changDict = {"4": "加保", "2": "退保", "3": "調保"}
web = 'https://eservice.nhi.gov.tw/nhiweb1/system/mLoginCa.aspx'

with open('Email.txt') as f:
    mails = f.read().split('\n')
while "" in mails:
    mails.remove("")
mails = set(mails)


def mailTo(title, mailAdds, message, whoSend='AutoDownloadList'):
    msg = MIMEMultipart()
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text = msg.as_string()
    # print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1', 25)
    s.sendmail(sender, mailAdds, text)
    s.quit()


def downloadAssureList():
    def renamefile(path):
        for pathTuple in os.walk(path):
            for file_ in pathTuple[2]:
                try:
                    if 'csv' in file_ and 'crdownload' not in file_:
                        name = []
                        uno = ""
                        savepath = ""
                        with open(os.path.join(pathTuple[0], file_), "r") as f:
                            datas = f.readlines()[1].split(",")
                            uno = mapAlias[datas[1]]
                            date = datas[0].replace("/", "")
                            style = "健保"
                            savepath = path + "\\健保"
                            if savepath and not os.path.exists(savepath):
                                os.makedirs(savepath)
                        try:
                            # print(file_)
                            if uno:
                                os.rename(os.path.join(pathTuple[0], file_),
                                          os.path.join(savepath, uno + style + "_" + date + ".csv"))
                        except Exception as er:
                            print(er)
                            print(os.path.join(path, file_) + "\t已有變更檔名的檔案")
                            os.remove(os.path.join(pathTuple[0], file_))
                except Exception as er:
                    report.append("更名失敗>>{},錯誤:{},出錯位置:{}".format(os.path.join(pathTuple[0], file_), er,
                                                                  sys.exc_info()[2].tb_lineno))

    report = []
    b = time.time()
    errorMsg = ""
    num = ""
    pinNum = e3.get()  # pin號碼
    chrome_options = Options()
    # crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔
    # chrome_options.add_extension('extension_0_0_1_3.crx')

    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": e2.get(),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })

    # webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
    # 套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon

    try:
        driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
        driver.get(web)

        dataDir = e1.get()
        inputPW = driver.find_element_by_id("USR_PASSWORD1")
        inputPW.clear()
        inputPW.send_keys(pinNum)
        driver.find_element_by_name("btnSubmit").click()

        alertText = ""
        try:
            x = driver.switch_to_alert()
            alertText = x.text
        except:
            pass
        if "其他錯誤" in alertText or "錯誤" in alertText:
            errorMsg = "密碼錯誤"
            print("密碼錯誤-登入失敗")
        else:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'UNIT_ID')))

        time.sleep(1)
        pathList = [i[0] + "\\" + j for i in os.walk(dataDir) for j in i[2] if 'xlsx' in j and "$" not in j]
        for excelFilePath in pathList:
            data = xlrd.open_workbook(excelFilePath)
            table = data.sheet_by_index(0)
            for i in range(10):
                try:
                    for text in table.col_values(i)[:7]:
                        #                 print(text+"---")
                        try:
                            if '店別' == text:
                                #                         print('xxx')
                                mapAliass = table.col_values(i)
                                aliasNumberList = [str(int(no)) for no in mapAliass if
                                                   no != "" and (type(no) == float or type(no) == int)]
                                #                                 print(aliasNumberList)
                            if '健保投保單位代號' == text:
                                mapUno = table.col_values(i)
                                print(text)
                                unoNumberList = [str(int(no)) for no in mapUno if
                                                 no != "" and type(no) == float]
                                #                                 print(unoNumberList)
                                break
                            else:
                                continue
                        except:
                            pass
                except:
                    pass

        mapAlias = {str(int(alias)): str(int(uno)) for uno, alias in zip(mapAliass, mapUno) if
                    uno != "" and (type(uno) == float or type(uno) == int)
                    and alias != "" and type(alias) == float}

        print(mapAlias)

        n = 0
        print("共有{}筆證號要下載".format(len(mapAlias)))
        while n < 2:
            roundTime = 0
            for num in mapAlias:
                try:
                    driver.find_element_by_id("UNIT_ID").click()
                    time.sleep(0.2)
                    driver.find_element_by_xpath('//option[@value="' + num + '"]').click()
                    driver.find_element_by_name("btnSubmit").click()
                    alertText = ""
                    try:
                        x = driver.switch_to_alert()
                        alertText = x.text
                        x.accept()
                    except:
                        pass
                    if "系統效期" in alertText or "單位使用本系統效期" in alertText:
                        if n == 0:
                            report.append("健保號碼:{}_沒權限無法登入".format(num))
                        continue

                    time.sleep(0.2)
                    driver.switch_to_frame(0)
                    try:
                        if driver.find_element_by_id("MenuMaker1_TreeView1t30").text == "":
                            viewNum = 31
                        else:
                            viewNum = 30
                    except:
                        if num == "134862801":
                            viewNum = 29
                        elif num == "144619018":
                            viewNum = 14
                    if '下載' not in driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).text:
                        if num == "134862801":
                            viewNum = 29
                        elif num == "144619018":
                            viewNum = 14



                    driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).click()
                    if n == 0:
                        driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 1)).click()
                        time.sleep(1)
                        driver.switch_to_frame(0)
                        driver.find_element_by_xpath('//option[@value="02"]').click()
                        time.sleep(0.3)
                        driver.find_element_by_id("ToolBar1_btnAdd").click()
                        time.sleep(1)
                        x = driver.switch_to_alert()
                        time.sleep(0.4)
                        try:
                            x.accept()
                        except:
                            pass
                        print("{}申請完畢".format(num))
                    else:
                        roundTime += 1
                        if roundTime % 30 == 0:
                            renamefile(e2.get())
                        driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 2)).click()
                        time.sleep(1)
                        driver.switch_to_frame(0)
                        try:
                            driver.find_element_by_id("DropDownList1").click()
                        except:
                            pass
                        driver.find_element_by_xpath('//option[@value="02"]').click()
                        time.sleep(0.4)
                        driver.find_element_by_id("ToolBar1_btnQuery").click()
                        time.sleep(0.8)
                        try:
                            driver.find_element_by_name("DataGrid1$ctl02$ctl00").click()
                        except:
                            driver.find_element_by_name("DataGrid1$ctl03$ctl00").click()
                        time.sleep(0.3)
                        print("{}下載完畢".format(num))
                    driver.switch_to_default_content()
                    time.sleep(0.1)
                    driver.find_element_by_class_name("Logout").click()
                    time.sleep(0.5)
                except Exception as er:
                    report.append("健保清冊下載失敗-證號:{},錯誤訊息:{},錯誤位置:{}".format(num, er, sys.exc_info()[2].tb_lineno))
                    try:
                        x = driver.switch_to_alert()
                        x.accept()
                    except:
                        pass
                    try:
                        driver.switch_to_default_content()
                    except:
                        pass
                    try:
                        driver.find_element_by_class_name("Logout").click()
                    except:
                        pass
                    time.sleep(0.5)
            if n == 0:
                print("已申請完畢，開始等待30秒")
                for i in range(30):
                    time.sleep(1)
                    print("剩餘{}秒".format(30 - i))
            n += 1

        renamefile(e2.get())

        e = time.time()
        report.append("耗時{}秒".format(round(e - b)))
        report.append("共處理{}筆".format(len(mapAlias)))
        mailTo('ReportDownList', mails, web + "\n" + "\n".join(report))
        driver.quit()

    except Exception as er:
        if errorMsg == "":
            errorMsg = str(er) + "\n證號:{}".format(
                num) + '\nPlease verify website connection \n if no problem in website find andy.yuan@wowprime.com \n 出錯位置:' + str(
                sys.exc_info()[2].tb_lineno)

        print('出錯了RRRRRRRRRRRR' + str(er) + '\n')

        mailTo('ReportError', mails, errorMsg)


def selectPath():
    path_ = askdirectory()
    path.set(path_)


def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)


with open('defaultIO.txt') as f:
    IO = f.read()
defaultLoadDir = IO.split("\n")[0].split("=")[1].replace("\\", "/")
defaultSaveDir = IO.split("\n")[1].split("=")[1].replace("\\", "/")
root = Tk()
path = StringVar()
Label(root, text="讀取資料夾:").grid(row=0, column=0)
e1 = Entry(root, textvariable=path)
e1.insert(0, defaultLoadDir)
e1.grid(row=0, column=1)
Button(root, text="讀取資料夾", command=selectPath).grid(row=0, column=2)

path2 = StringVar()
Label(root, text="存放資料夾:").grid(row=1, column=0)
e2 = Entry(root, textvariable=path2)
e2.insert(0, defaultSaveDir)
e2.grid(row=1, column=1)

Button(root, text="存放資料夾", command=selectPath2).grid(row=1, column=2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column=0)
e3 = Entry(root)
e3.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='申請健保清冊開始', command=downloadAssureList).grid(row=3, column=1, pady=4)
Button(root, text='下載健保清冊開始', command=downloadAssureList).grid(row=3, column=2, pady=4)
root.mainloop()


# 分開申請跟下載 健保清冊

In [7]:
from tkinter import Tk, StringVar, Label, Entry, Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait  # 直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC  # 判斷某物件存在，直到時間結束
import os
import sys
import time
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage

changDict = {"4": "加保", "2": "退保", "3": "調保"}
web = 'https://eservice.nhi.gov.tw/nhiweb1/system/mLoginCa.aspx'

with open('Email.txt') as f:
    mails = f.read().split('\n')
while "" in mails:
    mails.remove("")
mails = set(mails)


def mailTo(title, mailAdds, message, whoSend='AutoDownloadList'):
    msg = MIMEMultipart()
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text = msg.as_string()
    # print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1', 25)
    s.sendmail(sender, mailAdds, text)
    s.quit()


def applyAssureList():
    report = []
    b = time.time()
    errorMsg = ""
    num = ""
    pinNum = e3.get()  # pin號碼
    chrome_options = Options()
    # crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔
    # chrome_options.add_extension('extension_0_0_1_3.crx')

    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": e2.get(),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })

    # webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
    # 套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon

    try:
        driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
        driver.get(web)

        dataDir = e1.get()
        inputPW = driver.find_element_by_id("USR_PASSWORD1")
        inputPW.clear()
        inputPW.send_keys(pinNum)
        driver.find_element_by_name("btnSubmit").click()

        alertText = ""
        try:
            x = driver.switch_to_alert()
            alertText = x.text
        except:
            pass
        if "其他錯誤" in alertText or "錯誤" in alertText:
            errorMsg = "密碼錯誤"
            print("密碼錯誤-登入失敗")
        else:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'UNIT_ID')))

        time.sleep(1)
        pathList = [i[0] + "\\" + j for i in os.walk(dataDir) for j in i[2] if 'xlsx' in j and "$" not in j]
        for excelFilePath in pathList:
            data = xlrd.open_workbook(excelFilePath)
            table = data.sheet_by_index(0)
            for i in range(10):
                try:
                    for text in table.col_values(i)[:7]:
                        #                 print(text+"---")
                        try:
                            if '店別' == text:
                                #                         print('xxx')
                                mapAliass = table.col_values(i)
                                aliasNumberList = [str(int(no)) for no in mapAliass if
                                                   no != "" and (type(no) == float or type(no) == int)]
                                #                                 print(aliasNumberList)
                            if '健保投保單位代號' == text:
                                mapUno = table.col_values(i)
                                print(text)
                                unoNumberList = [str(int(no)) for no in mapUno if
                                                 no != "" and type(no) == float]
                                #                                 print(unoNumberList)
                                break
                            else:
                                continue
                        except:
                            pass
                except:
                    pass

        mapAlias = {str(int(alias)): str(int(uno)) for uno, alias in zip(mapAliass, mapUno) if
                    uno != "" and (type(uno) == float or type(uno) == int)
                    and alias != "" and type(alias) == float}

        print(mapAlias)

        n = 0
        print("共有{}筆證號要下載".format(len(mapAlias)))
        roundTime = 0
        for num in mapAlias:
            try:
                driver.find_element_by_id("UNIT_ID").click()
                time.sleep(0.2)
                driver.find_element_by_xpath('//option[@value="' + num + '"]').click()
                driver.find_element_by_name("btnSubmit").click()
                alertText = ""
                try:
                    x = driver.switch_to_alert()
                    alertText = x.text
                    x.accept()
                except:
                    pass
                if "系統效期" in alertText or "單位使用本系統效期" in alertText:
                    if n == 0:
                        report.append("健保號碼:{}_沒權限無法登入".format(num))
                    continue

                time.sleep(0.2)
                driver.switch_to_frame(0)
                try:
                    if driver.find_element_by_id("MenuMaker1_TreeView1t30").text == "":
                        viewNum = 31
                    else:
                        viewNum = 30
                except:
                    if num == "134862801":
                        viewNum = 29
                    elif num == "144619018":
                        viewNum = 14
                if '下載' not in driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).text:
                    if num == "134862801":
                        viewNum = 29
                    elif num == "144619018":
                        viewNum = 14



                driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).click()
                if n == 0:
                    driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 1)).click()
                    time.sleep(1)
                    driver.switch_to_frame(0)
                    driver.find_element_by_xpath('//option[@value="02"]').click()
                    time.sleep(0.3)
                    driver.find_element_by_id("ToolBar1_btnAdd").click()
                    time.sleep(1)
                    x = driver.switch_to_alert()
                    time.sleep(0.4)
                    try:
                        x.accept()
                    except:
                        pass
                    print("{}申請完畢".format(num))
                else:
                    roundTime += 1
                    if roundTime % 30 == 0:
                        renamefile(e2.get())
                    driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 2)).click()
                    time.sleep(1)
                    driver.switch_to_frame(0)
                    try:
                        driver.find_element_by_id("DropDownList1").click()
                    except:
                        pass
                    driver.find_element_by_xpath('//option[@value="02"]').click()
                    time.sleep(0.4)
                    driver.find_element_by_id("ToolBar1_btnQuery").click()
                    time.sleep(0.8)
                    try:
                        driver.find_element_by_name("DataGrid1$ctl02$ctl00").click()
                    except:
                        driver.find_element_by_name("DataGrid1$ctl03$ctl00").click()
                    time.sleep(0.3)
                    print("{}下載完畢".format(num))
                driver.switch_to_default_content()
                time.sleep(0.1)
                driver.find_element_by_class_name("Logout").click()
                time.sleep(0.5)
            except Exception as er:
                report.append("健保清冊申請失敗-證號:{},錯誤訊息:{},錯誤位置:{}".format(num, er, sys.exc_info()[2].tb_lineno))
                try:
                    x = driver.switch_to_alert()
                    x.accept()
                except:
                    pass
                try:
                    driver.switch_to_default_content()
                except:
                    pass
                try:
                    driver.find_element_by_class_name("Logout").click()
                except:
                    pass
                time.sleep(0.5)
        e = time.time()
        print("申請完畢")
        report.append("耗時{}秒".format(round(e - b)))
        report.append("共處理{}筆申請".format(len(mapAlias)))
        mailTo('ReportApplyList', mails, web + "\n" + "\n".join(report))
        driver.quit()

    except Exception as er:
        if errorMsg == "":
            errorMsg = str(er) + "\n證號:{}".format(
                num) + '\nPlease verify website connection \n if no problem in website find andy.yuan@wowprime.com \n 出錯位置:' + str(
                sys.exc_info()[2].tb_lineno)

        print('出錯了RRRRRRRRRRRR' + str(er) + '\n')

        mailTo('ReportError', mails, errorMsg)


        
        
def downloadAssureList():
    def renamefile(path):
        for pathTuple in os.walk(path):
            for file_ in pathTuple[2]:
                try:
                    if 'csv' in file_ and 'crdownload' not in file_:
                        name = []
                        uno = ""
                        savepath = ""
                        with open(os.path.join(pathTuple[0], file_), "r") as f:
                            datas = f.readlines()[1].split(",")
                            uno = mapAlias[datas[1]]
                            date = datas[0].replace("/", "")
                            style = "健保"
                            savepath = path + "\\健保"
                            if savepath and not os.path.exists(savepath):
                                os.makedirs(savepath)
                        try:
                            # print(file_)
                            if uno:
                                os.rename(os.path.join(pathTuple[0], file_),
                                          os.path.join(savepath, uno + style + "_" + date + ".csv"))
                        except Exception as er:
                            print(er)
                            print(os.path.join(path, file_) + "\t已有變更檔名的檔案")
                            os.remove(os.path.join(pathTuple[0], file_))
                except Exception as er:
                    report.append("更名失敗>>{},錯誤:{},出錯位置:{}".format(os.path.join(pathTuple[0], file_), er,
                                                                  sys.exc_info()[2].tb_lineno))

    report = []
    b = time.time()
    errorMsg = ""
    num = ""
    pinNum = e3.get()  # pin號碼
    chrome_options = Options()
    # crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔
    # chrome_options.add_extension('extension_0_0_1_3.crx')

    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": e2.get(),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })

    # webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
    # 套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon

    try:
        driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
        driver.get(web)

        dataDir = e1.get()
        inputPW = driver.find_element_by_id("USR_PASSWORD1")
        inputPW.clear()
        inputPW.send_keys(pinNum)
        driver.find_element_by_name("btnSubmit").click()

        alertText = ""
        try:
            x = driver.switch_to_alert()
            alertText = x.text
        except:
            pass
        if "其他錯誤" in alertText or "錯誤" in alertText:
            errorMsg = "密碼錯誤"
            print("密碼錯誤-登入失敗")
        else:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'UNIT_ID')))

        time.sleep(1)
        pathList = [i[0] + "\\" + j for i in os.walk(dataDir) for j in i[2] if 'xlsx' in j and "$" not in j]
        for excelFilePath in pathList:
            data = xlrd.open_workbook(excelFilePath)
            table = data.sheet_by_index(0)
            for i in range(10):
                try:
                    for text in table.col_values(i)[:7]:
                        #                 print(text+"---")
                        try:
                            if '店別' == text:
                                #                         print('xxx')
                                mapAliass = table.col_values(i)
                                aliasNumberList = [str(int(no)) for no in mapAliass if
                                                   no != "" and (type(no) == float or type(no) == int)]
                                #                                 print(aliasNumberList)
                            if '健保投保單位代號' == text:
                                mapUno = table.col_values(i)
                                print(text)
                                unoNumberList = [str(int(no)) for no in mapUno if
                                                 no != "" and type(no) == float]
                                #                                 print(unoNumberList)
                                break
                            else:
                                continue
                        except:
                            pass
                except:
                    pass

        mapAlias = {str(int(alias)): str(int(uno)) for uno, alias in zip(mapAliass, mapUno) if
                    uno != "" and (type(uno) == float or type(uno) == int)
                    and alias != "" and type(alias) == float}

        print(mapAlias)

        n = 1
        print("共有{}筆證號要下載".format(len(mapAlias)))
        roundTime = 0
        for num in mapAlias:
            try:
                driver.find_element_by_id("UNIT_ID").click()
                time.sleep(0.2)
                driver.find_element_by_xpath('//option[@value="' + num + '"]').click()
                driver.find_element_by_name("btnSubmit").click()
                alertText = ""
                try:
                    x = driver.switch_to_alert()
                    alertText = x.text
                    x.accept()
                except:
                    pass
                if "系統效期" in alertText or "單位使用本系統效期" in alertText:
                    if n == 0:
                        report.append("健保號碼:{}_沒權限無法登入".format(num))
                    continue

                time.sleep(0.2)
                driver.switch_to_frame(0)
                try:
                    if driver.find_element_by_id("MenuMaker1_TreeView1t30").text == "":
                        viewNum = 31
                    else:
                        viewNum = 30
                except:
                    if num == "134862801":
                        viewNum = 29
                    elif num == "144619018":
                        viewNum = 14
                if '下載' not in driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).text:
                    if num == "134862801":
                        viewNum = 29
                    elif num == "144619018":
                        viewNum = 14



                driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).click()
                if n == 0:
                    driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 1)).click()
                    time.sleep(1)
                    driver.switch_to_frame(0)
                    driver.find_element_by_xpath('//option[@value="02"]').click()
                    time.sleep(0.3)
                    driver.find_element_by_id("ToolBar1_btnAdd").click()
                    time.sleep(1)
                    x = driver.switch_to_alert()
                    time.sleep(0.4)
                    try:
                        x.accept()
                    except:
                        pass
                    print("{}申請完畢".format(num))
                else:
                    roundTime += 1
                    if roundTime % 30 == 0:
                        renamefile(e2.get())
                    driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 2)).click()
                    time.sleep(1)
                    driver.switch_to_frame(0)
                    try:
                        driver.find_element_by_id("DropDownList1").click()
                    except:
                        pass
                    driver.find_element_by_xpath('//option[@value="02"]').click()
                    time.sleep(0.4)
                    driver.find_element_by_id("ToolBar1_btnQuery").click()
                    time.sleep(0.8)
                    try:
                        driver.find_element_by_name("DataGrid1$ctl02$ctl00").click()
                    except:
                        driver.find_element_by_name("DataGrid1$ctl03$ctl00").click()
                    time.sleep(0.3)
                    print("{}下載完畢".format(num))
                driver.switch_to_default_content()
                time.sleep(0.1)
                driver.find_element_by_class_name("Logout").click()
                time.sleep(0.5)
            except Exception as er:
                report.append("健保清冊下載失敗-證號:{},錯誤訊息:{},錯誤位置:{}".format(num, er, sys.exc_info()[2].tb_lineno))
                try:
                    x = driver.switch_to_alert()
                    x.accept()
                except:
                    pass
                try:
                    driver.switch_to_default_content()
                except:
                    pass
                try:
                    driver.find_element_by_class_name("Logout").click()
                except:
                    pass
                time.sleep(0.5)
        renamefile(e2.get())

        e = time.time()
        print("下載完畢")
        report.append("耗時{}秒".format(round(e - b)))
        report.append("共處理{}筆下載".format(len(mapAlias)))
        mailTo('ReportDownList', mails, web + "\n" + "\n".join(report))
        driver.quit()

    except Exception as er:
        if errorMsg == "":
            errorMsg = str(er) + "\n證號:{}".format(
                num) + '\nPlease verify website connection \n if no problem in website find andy.yuan@wowprime.com \n 出錯位置:' + str(
                sys.exc_info()[2].tb_lineno)

        print('出錯了RRRRRRRRRRRR' + str(er) + '\n')

        mailTo('ReportError', mails, errorMsg)        
        
        
def selectPath():
    path_ = askdirectory()
    path.set(path_)


def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)


with open('defaultIO.txt') as f:
    IO = f.read()
defaultLoadDir = IO.split("\n")[0].split("=")[1].replace("\\", "/")
defaultSaveDir = IO.split("\n")[1].split("=")[1].replace("\\", "/")
root = Tk()
path = StringVar()
Label(root, text="讀取資料夾:").grid(row=0, column=0)
e1 = Entry(root, textvariable=path)
e1.insert(0, defaultLoadDir)
e1.grid(row=0, column=1)
Button(root, text="讀取資料夾", command=selectPath).grid(row=0, column=2)

path2 = StringVar()
Label(root, text="存放資料夾:").grid(row=1, column=0)
e2 = Entry(root, textvariable=path2)
e2.insert(0, defaultSaveDir)
e2.grid(row=1, column=1)

Button(root, text="存放資料夾", command=selectPath2).grid(row=1, column=2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column=0)
e3 = Entry(root)
e3.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='申請健保清冊開始', command=applyAssureList).grid(row=3, column=1, pady=4)
Button(root, text='下載健保清冊開始', command=downloadAssureList).grid(row=3, column=2, pady=4)
root.mainloop()


健保投保單位代號
{'145345099': '11801', '145345080': '11802'}
共有2筆證號要下載
145345099申請完畢
145345080申請完畢
申請完畢
健保投保單位代號
{'145345099': '11801', '145345080': '11802'}
共有2筆證號要下載
145345099下載完畢
145345080下載完畢
下載完畢


In [10]:
import pymongo 
client=pymongo.MongoClient("localhost",27017,username='j122085',password='850605')

In [14]:
x=list(client.rawData.ipeenInfo.find({}))

In [19]:
list({i['bigstyle'] for i in x})

['其他美食',
 '咖啡、簡餐、茶',
 '烘焙、甜點、零食',
 '',
 'buffet自助餐',
 '中式料理',
 '主題特色餐廳',
 '早餐',
 '亞洲料理',
 '冰品、飲料、甜湯',
 '小吃',
 '鍋類',
 '素食',
 '異國料理',
 '日式料理',
 '速食料理',
 '燒烤類']